In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session, flash, get_flashed_messages
from flask_ngrok import run_with_ngrok
import sqlite3
from werkzeug.security import generate_password_hash, check_password_hash
import nltk
nltk.download('punkt')
from fuzzywuzzy import fuzz
import heapq

app = Flask(__name__)
run_with_ngrok(app)
app.secret_key = 'DIEf_only'

def get_db():
    conn = sqlite3.connect('book_recommendation.db')
    return conn

def close_db(conn):
    conn.close()

def user_authenticated():
    return 'user_id' in session

@app.route('/register', methods=['GET', 'POST'])
def register():
    get_flashed_messages()  # Add this line to clear any existing flash messages
    if request.method == 'POST':
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']

        flash('Registration successful! You can now log in.', 'success')
        return redirect('/login')

    return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        conn = get_db()
        cursor = conn.cursor()

        cursor.execute('SELECT user_id, password_hash FROM users WHERE username = ?', (username,))
        user = cursor.fetchone()

        if user is None or not check_password_hash(user[1], password):
            flash('Incorrect username or password.', 'danger')
            return redirect(url_for('login'))

        session['user_id'] = user[0]  # Set 'user_id' in session

        flash('Login successful!', 'success')

        return redirect(url_for('user_menu'))

    return render_template('login.html')

def calculate_similarity(title1, title2):
    words1 = set(nltk.word_tokenize(title1[0].lower()))
    words2 = set(nltk.word_tokenize(title2.lower()))
    return fuzz.token_set_ratio(words1, words2)

@app.route('/search', methods=['POST'])
def search():
    if not user_authenticated():
        return redirect(url_for('main_menu'))

    query = request.form.get('query', '')

    db = get_db()
    cursor = db.cursor()

    query_string = """
        SELECT title, author, categories 
        FROM books 
        WHERE title LIKE ? OR author LIKE ?
        ORDER BY CASE
            WHEN author LIKE ? THEN 1
            WHEN title LIKE ? THEN 2
            ELSE 3
        END, title
        LIMIT 15
    """
    cursor.execute(query_string, (f"%{query}%", f"%{query}%", f"{query}%", f"{query}%"))
    search_results = cursor.fetchall()
    close_db(db)

    if search_results:
        session['last_searched_category'] = search_results[0][2]

    return render_template('search_results.html', search_results=search_results)

@app.route('/recommend')
def recommend():
    if not user_authenticated():
        return redirect(url_for('main_menu'))

    user_id = session['user_id']
    book_title = request.args.get('book_title', '')

    conn = get_db()
    cursor = conn.cursor()

    cursor.execute('SELECT title, author FROM books')
    all_books = cursor.fetchall()

    scores = [((title, author), calculate_similarity((title,), book_title)) for title, author in all_books]
    recommended_books = heapq.nlargest(10, scores, key=lambda x: x[1])

    # Create a new list of tuples that only contain the title and author
    recommended_books = [book[0] for book in recommended_books]

    close_db(conn)

    return render_template('recommendations.html', recommended_books=recommended_books)

@app.route('/')
def main_menu():
    return render_template('main_menu.html')

@app.route('/user_menu')
def user_menu():
    if not user_authenticated():
        return redirect(url_for('main_menu'))
    return render_template('user_menu.html')

@app.route('/search_page')
def search_page():
    if not user_authenticated():
        return redirect(url_for('main_menu'))
    return render_template('search.html')

@app.route('/logout', methods=['GET', 'POST'])
def logout():
    session.clear()
    flash('You have been logged out.', 'success')
    return redirect(url_for('main_menu'))

if __name__ == '__main__':
    app.run()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Domin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


 * Running on http://bd09-158-129-238-236.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Dec/2023 13:36:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 13:36:30] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 13:36:40] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [01/Dec/2023 13:36:40] "GET /user_menu HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 13:36:51] "GET /search_page HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 13:36:57] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 13:37:03] "GET /user_menu HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 13:37:22] "GET /recommend?book_title=Chamber+of+secrets HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 13:37:38] "GET /user_menu HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 13:37:41] "GET /search_page HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 13:37:57] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 13:38:18] "GET /search_page HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 13:38:23] "GET /user_menu HTTP/1.1" 200 -
